- **Author**: LIKE NING
- **Manuscript**: 
- **Creation date**: 2025-01-16
- **Latest update**: 2025-01-16 \
This Jupyter notebook produces Fig. spmaxfrzthw of the manuscript.

### 1.综合计算
#### 1.1 albedo mutiply sunshine hours

In [ ]:
%%bash
# #!/bin/bash
albedo_dir="/mnt/syn/tian/1_albedo/update/albedo_update"
shr_dir="/mnt/syn/tian/1_albedo/shr"
alpha_dir="/mnt/syn/tian/1_albedo/update/alpha_update"
rf_dir="/mnt/syn/tian/1_albedo/update/rf_update"
gwi_dir="/mnt/syn/tian/1_albedo/update/gwi_update"
mlist="/mnt/qnap/tian/tibet/alpha_date.txt"
rflist="/mnt/qnap/tian/tibet/rf_scalefactor.txt"
swlist="/mnt/qnap/tian/tibet/swin.txt"
mkdir -p $alpha_dir $rf_dir $gwi_dir

# 从文件中读取每一行数据到列表中
mylist=($(cat $mlist))
rfslist=($(cat $rflist))
swslist=($(cat $swlist))

# for i in $(seq 0 7304);do 
for i in $(seq 0 366);do 
    date=${mylist[$i]}
    swin=${swslist[$i]}
    rfsc=${rfslist[$i]}
    # echo $swin
    rfsc=${rfslist[$i]}
    albedo_nc=$albedo_dir/a_$date.nc
    shr_nc=$shr_dir/sh$date.nc
    alpha_nc=$alpha_dir/alpha_$date.nc
    rf_nc=$rf_dir/rf_$date.nc
    gwi_nc=$gwi_dir/gwi_$date.nc
    # echo $out_nc
    cdo -b F64 -O -mul -divc,1000 -setmissval,-9999 $albedo_nc -setmissval,-9999 $shr_nc $alpha_nc
    cdo -b F64 -O -mulc,$swin -mul -divc,1000 -setmissval,-9999 $albedo_nc -setmissval,-9999 $shr_nc $rf_nc
    cdo -b F64 -O -mulc,$rfsc -mul -divc,1000 -setmissval,-9999 $albedo_nc -setmissval,-9999 $shr_nc $gwi_nc
done

In [ ]:
%%bash
# #!/bin/bash
albedo_dir="/mnt/syn/tian/1_albedo/albedo"
shr_dir="/mnt/syn/tian/1_albedo/shr"
gwi_dir="/mnt/syn/tian/1_albedo/new/gwi_new"
mlist="/mnt/qnap/tian/tibet/alpha_date.txt"
rflist="/mnt/qnap/tian/tibet/rf_scalefactor.txt"
mkdir -p $gwi_dir

# 从文件中读取每一行数据到列表中
mylist=($(cat $mlist))
rfslist=($(cat $rflist))

for i in $(seq 0 7304);do 
    date=${mylist[$i]}
    rfsc=${rfslist[$i]}
    albedo_nc=$albedo_dir/a_$date.nc
    shr_nc=$shr_dir/sh$date.nc
    out_nc=$gwi_dir/gwi_$date.nc
    echo $out_nc
    cdo -b F64 -O -mulc,$rfsc -mul -divc,1000 -setmissval,-9999 $albedo_nc -setmissval,-9999 $shr_nc $out_nc
done

#### 1.2 合并为逐年文件

In [ ]:
%%bash
# #!/bin/bash
albedo_dir="/mnt/syn/tian/1_albedo/update/albedo_update"
albedo_yr_dir="/mnt/syn/tian/1_albedo/update/albedo_yr_update"
mlist="/mnt/qnap/tian/tibet/alpha_yr.txt"
mkdir -p $albedo_yr_dir

# 从文件中读取每一行数据到列表中
mylist=($(cat $mlist))

# 遍历列表中的数据
for date in ${mylist[@]}
do
    echo $date
    albedo_nc=$albedo_dir/a_$date*.nc
    out_nc=$albedo_yr_dir/albedo_$date.nc
    cdo -settaxis,${date}-01-01,00:00:00,1day -cat $albedo_nc $out_nc
done


#### 1.3 作差计算(直接使用日数据作差)

In [ ]:
%%bash
# #!/bin/bash
gwi_yr_dir="/mnt/syn/tian/1_albedo/new/gwi_yr"
gwi_delta_dir="/mnt/syn/tian/1_albedo/new/gwi_delta"
mlist="/mnt/qnap/tian/tibet/alpha_yr.txt"
mkdir -p $gwi_delta_dir

basegwi="/mnt/syn/tian/1_albedo/new/gwi_yr/gwi_2001.nc"
# 从文件中读取每一行数据到列表中
mylist=($(cat $mlist))

# 遍历列表中的数据
for date in ${mylist[@]}
do
    echo $date
    gwi_nc=$gwi_yr_dir/gwi_$date*.nc
    out_nc=$gwi_delta_dir/deltagwi_$date.nc
    cdo -O -sub $basegwi $gwi_nc $out_nc
done

### 1.4 作差计算（使用月数据作差）

In [ ]:
%%bash
# #!/bin/bash
gwi_yr_dir="/mnt/syn/tian/1_albedo/new/gwi_yr"
gwi_delta_dir="/mnt/syn/tian/1_albedo/new/gwi_delta"
mlist="/mnt/qnap/tian/tibet/alpha_yr.txt"
mkdir -p $gwi_delta_dir

basegwi="/mnt/syn/tian/1_albedo/new/gwi_yr/gwi_2001.nc"
# 从文件中读取每一行数据到列表中
mylist=($(cat $mlist))

# 遍历列表中的数据
for date in ${mylist[@]}
do
    echo $date
    gwi_nc=$gwi_yr_dir/gwi_$date*.nc
    out_nc=$gwi_delta_dir/deltagwi_mon_$date.nc
    cdo -O -sub -monmean $basegwi -monmean $gwi_nc $out_nc
done

## 修改文件属性（年份）

In [ ]:
%%bash
# #!/bin/bash
gwi_delta_dir="/mnt/syn/tian/1_albedo/new/gwi_delta"
mlist="/mnt/qnap/tian/tibet/alpha_yr.txt"
mkdir -p $gwi_delta_dir


# 从文件中读取每一行数据到列表中
mylist=($(cat $mlist))

# 遍历列表中的数据
for date in ${mylist[@]}
do
    echo $date
    deltagwi_nc=$gwi_delta_dir/deltagwi_mon_$date.nc
    deltagwi_nc_new=$gwi_delta_dir/deltagwi_mon_new_$date.nc
    cdo settaxis,$date-01-15,00:00,1months $deltagwi_nc $deltagwi_nc_new
    ncatted -O -h -a history,global,d,, $deltagwi_nc_new $deltagwi_nc
    rm -rf $deltagwi_nc
    mv $deltagwi_nc_new $deltagwi_nc
    # rm -rf $deltagwi_nc_new
done

In [ ]:
%%bash
# #!/bin/bash
albedo_dir="/mnt/syn/tian/1_albedo/albedo"
albedo_delta_dir="/mnt/syn/tian/1_albedo/new/albedo_delta"
mlist="/mnt/qnap/tian/tibet/alpha_date.txt"
rflist="/mnt/qnap/tian/tibet/rf_scalefactor.txt"
mkdir -p $gwi_dir

# 从文件中读取每一行数据到列表中
mylist=($(cat $mlist))
rfslist=($(cat $rflist))

for i in $(seq 0 7304);do 
    date=${mylist[$i]}
    rfsc=${rfslist[$i]}
    albedo_nc=$albedo_dir/a_$date.nc
    out_nc=$albedo_delta_dir/deltalbedo_$date.nc
    echo $out_nc
    cdo -b F64 -O -mulc,$rfsc -mul -divc,1000 -setmissval,-9999 $albedo_nc -setmissval,-9999 $shr_nc $out_nc
done

In [ ]:
%%bash
# #!/bin/bash
albedo_dir="/mnt/syn/tian/1_albedo/albedo"
albedo_yr_dir="/mnt/syn/tian/1_albedo/new/albedo_yr_new"
mlist="/mnt/qnap/tian/tibet/alpha_yr.txt"
mkdir -p $albedo_yr_dir

# 从文件中读取每一行数据到列表中
mylist=($(cat $mlist))

# 遍历列表中的数据
for date in ${mylist[@]}
do
    echo $date
    albedo_nc=$albedo_dir/a_$date*.nc
    out_nc=$albedo_yr_dir/albedo_$date.nc
    cdo -settaxis,${date}-01-01,00:00:00,1day -cat $albedo_nc $out_nc
done

In [ ]:
%%bash
# #!/bin/bash
albedo_yr_dir="/mnt/syn/tian/1_albedo/new/albedo_yr_new"
albedo_delta_dir="/mnt/syn/tian/1_albedo/new/albedo_delta_new"
mlist="/mnt/qnap/tian/tibet/alpha_yr.txt"
mkdir -p $albedo_delta_dir

basegwi="/mnt/syn/tian/1_albedo/new/albedo_yr_new/albedo_2001.nc"
# 从文件中读取每一行数据到列表中
mylist=($(cat $mlist))

# 遍历列表中的数据
for date in ${mylist[@]}
do
    echo $date
    albedo_nc=$albedo_yr_dir/albedo_$date.nc
    out_nc=$albedo_delta_dir/deltalbedo_mon_$date.nc
    cdo -O -sub -monmean $basegwi -monmean $albedo_nc $out_nc
done